In [1]:
import os
import io
import pandas as pd
import tensorflow as tf
import sys
from google.protobuf import text_format

In [2]:
#needs research/ in PYTHONPATH
sys.path.append('../models/research')
from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple, OrderedDict

In [ ]:
from glob import glob
#rescale images if needed
# WARNING: Only works for quadratic images currently
basewidth = 2291
for imgfile in glob(os.path.join(os.getcwd(), 'images/train', '*.png')):
    img = Image.open(imgfile)
    img = img.resize((basewidth, basewidth), Image.LANCZOS)
    img.save(imgfile)


In [3]:
flags = tf.app.flags
flags.DEFINE_string('csv_input', 'annotations/train_labels.csv', 'Path to the CSV input')
flags.DEFINE_string('output_path', 'annotations/train.record', 'Path to output TFRecord')
flags.DEFINE_string('img_path', 'images/train', 'Path to images')
flags.DEFINE_string('labelmap_path', 'annotations/label_map.pbtxt', 'Path to label_map.pbtxt')
FLAGS = flags.FLAGS

In [4]:
def class_text_to_int(row_label):
    label_map = label_map_util.load_labelmap(FLAGS.labelmap_path)
    categories = label_map_util.convert_label_map_to_categories(label_map,
     max_num_classes=label_map_util.get_max_label_map_index(label_map), use_display_name=True)
    for category in categories:
        if (row_label == category['name']):
            return category['id']
    return None



def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    # check if the image format is matching with your images.
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def main(_):
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    path = os.path.join(os.getcwd(), FLAGS.img_path)
    examples = pd.read_csv(FLAGS.csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), FLAGS.output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))


Create Tensorflow records
==

Set the `flags` and `paths` above correctly before running!
If you get `DuplicateError`: Restart IPython kernel!

In [5]:
tf.app.run()

Successfully created the TFRecords: /home/oskar/workspace/cjt_hazmat/hazmat/annotations/train.record


SystemExit: 

/home/oskar/workspace/cjt_hazmat/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
